# https://blog.csdn.net/firparks/article/details/54563683

### 基于以下语料建立语言模型
研究生物很有意思。

他大学时代是研究生物的。

生物专业是他的首选目标。

他是研究生。

### 尝试以“词”作为基元，计算出现句子“他是研究生物的”的概率

In [1]:
import jieba
from _overlapped import NULL

#将句子变为"BOSxxxxxEOS"这种形式

In [2]:
def reform(sentence):
    #如果是以“。”结束的则将“。”删掉
    if sentence.endswith("。"):
        sentence=sentence[:-1]
    #添加起始符BOS和终止符EOS   
    sentence_modify1=sentence.replace("。", "EOSBOS")
    sentence_modify2="BOS"+sentence_modify1+"EOS"
    print(sentence_modify2)
    return sentence_modify2

#分词并统计词频

In [3]:
def segmentation(sentence,lists,dicts=NULL):
    jieba.suggest_freq("BOS", True)
    jieba.suggest_freq("EOS", True)
    sentence = jieba.cut(sentence,HMM=False)
    format_sentence=",".join(sentence)
    #将词按","分割后依次填入数组word_list[]
    lists=format_sentence.split(",")
    #统计词频，如果词在字典word_dir{}中出现过则+1，未出现则=1
    if dicts!=NULL:
        for word in lists:
            if word not in dicts:
                dicts[word]=1
            else:
                dicts[word]+=1               
    return lists

#比较两个数列，二元语法

In [4]:
def compareList(ori_list,test_list):
    #申请空间
    count_list=[0]*(len(test_list))
    #遍历测试的字符串
    for i in range(0,len(test_list)-1):
        #遍历语料字符串，且因为是二元语法，不用比较语料字符串的最后一个字符
        for j in range(0,len(ori_list)-2):                
            #如果测试的第一个词和语料的第一个词相等则比较第二个词
            if test_list[i]==ori_list[j]:
                if test_list[i+1]==ori_list[j+1]:
                    count_list[i]+=1
    return count_list

#计算概率   ，加一平滑 

In [5]:
def probability(test_list,count_list,ori_dict):
    flag=0
    #概率值为p
    p=1
    for key in test_list: 
        #数据平滑处理：加1法
        p*=(float(count_list[flag]+1)/float(ori_dict[key]+1))
        flag+=1
    return p

#语料句子

In [6]:
sentence_ori="研究生物很有意思。他大学时代是研究生物的。生物专业是他的首选目标。他是研究生。"
ori_list=[]
ori_dict={}
sentence_ori_temp=""

#测试句子

In [7]:
sentence_test="他是研究生物的"
sentence_test_temp="" 
test_list=[]
count_list=[]
p=0

In [8]:
#分词并将结果存入一个list，词频统计结果存入字典

In [9]:
sentence_ori_temp=reform(sentence_ori)
ori_list=segmentation(sentence_ori_temp,ori_list,ori_dict)

sentence_test_temp=reform(sentence_test)
test_list=segmentation(sentence_test_temp,test_list)

count_list=compareList(ori_list, test_list)
p=probability(test_list,count_list,ori_dict)

print('"',sentence_test,'"出现的概率是：',p)



Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\David\AppData\Local\Temp\jieba.cache


BOS研究生物很有意思EOSBOS他大学时代是研究生物的EOSBOS生物专业是他的首选目标EOSBOS他是研究生EOS


Loading model cost 0.624 seconds.
Prefix dict has been built successfully.


BOS他是研究生物的EOS
" 他是研究生物的 "出现的概率是： 0.01


In [11]:
'''
Created on 2017年1月15日

@author: 薛沛雷
'''

import jieba
from _overlapped import NULL


#将句子变为"BOSxxxxxEOS"这种形式
def reform(sentence):
    #如果是以“。”结束的则将“。”删掉
    if sentence.endswith("。"):
        sentence=sentence[:-1]
    #添加起始符BOS和终止符EOS   
    sentence_modify1=sentence.replace("。", "EOSBOS")
    sentence_modify2="BOS"+sentence_modify1+"EOS"
    return sentence_modify2


#分词并统计词频
def segmentation(sentence,lists,dicts=NULL):
    jieba.suggest_freq("BOS", True)
    jieba.suggest_freq("EOS", True)
    sentence = jieba.cut(sentence,HMM=False)
    format_sentence=",".join(sentence)
    #将词按","分割后依次填入数组word_list[]
    lists=format_sentence.split(",")
    #统计词频，如果词在字典word_dir{}中出现过则+1，未出现则=1
    if dicts!=NULL:
        for word in lists:
            if word not in dicts:
                dicts[word]=1
            else:
                dicts[word]+=1               
    print('lists:',lists)
    print('dicts:',dicts)
    return lists


#比较两个数列，二元语法
def compareList(ori_list,test_list):
    #申请空间
    count_list=[0]*(len(test_list))
    #遍历测试的字符串
    for i in range(0,len(test_list)-1):
        #遍历语料字符串，且因为是二元语法，不用比较语料字符串的最后一个字符
        for j in range(0,len(ori_list)-2):                
            #如果测试的第一个词和语料的第一个词相等则比较第二个词
            if test_list[i]==ori_list[j]:
                if test_list[i+1]==ori_list[j+1]:
                    count_list[i]+=1
    return count_list


#计算概率    
def probability(test_list,count_list,ori_dict):
    flag=0
    #概率值为p
    p=1
    for key in test_list: 
        #数据平滑处理：加1法
        p*=(float(count_list[flag]+1)/float(ori_dict[key]+1))
        flag+=1
    return p


if __name__ == "__main__":

    #语料句子
    sentence_ori="研究生物很有意思。他大学时代是研究生物的。生物专业是他的首选目标。他是研究生。"
    ori_list=[]
    ori_dict={}
    sentence_ori_temp=""

    #测试句子
    sentence_test="他是研究生物的"
    sentence_test_temp="" 
    test_list=[]
    count_list=[]
    p=0

    #分词并将结果存入一个list，词频统计结果存入字典
    sentence_ori_temp=reform(sentence_ori)
    ori_list=segmentation(sentence_ori_temp,ori_list,ori_dict)

    sentence_test_temp=reform(sentence_test)
    test_list=segmentation(sentence_test_temp,test_list)

    count_list=compareList(ori_list, test_list)
    print('\ncount_list',count_list)
    p=probability(test_list,count_list,ori_dict)
    print(p)



lists: ['BOS', '研究', '生物', '很', '有意思', 'EOS', 'BOS', '他', '大学', '时代', '是', '研究', '生物', '的', 'EOS', 'BOS', '生物', '专业', '是', '他', '的', '首选', '目标', 'EOS', 'BOS', '他', '是', '研究生', 'EOS']
dicts: {'BOS': 4, '研究': 2, '生物': 3, '很': 1, '有意思': 1, 'EOS': 4, '他': 3, '大学': 1, '时代': 1, '是': 3, '的': 2, '专业': 1, '首选': 1, '目标': 1, '研究生': 1}
lists: ['BOS', '他', '是', '研究', '生物', '的', 'EOS']
dicts: 0

count_list [2, 1, 1, 2, 1, 1, 0]
0.01
